In [5]:
! pip show llama-index


Name: llama-index
Version: 0.11.10
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: C:\Users\0201132\Documents\Projects\text_to_sql_project\venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 


In [1]:
! pip install llama-index==0.6.20 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.114.2 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
llama-index-agent-openai 0.3.1 requires openai>=1.14.0, but you have openai 1.5.0 which is incompatible.
llama-index-core 0.11.10 requires pydantic<3.0.0,>=2.7.0, but you have pydantic 1.10.18 which is incompatible.
llama-index-llms-openai 0.2.7 requires openai<2.0.0,>=1.40.0, but you have openai 1.5.0 which is incompatible.
pydantic-core 2.23.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [1]:
from llama_index import (
    LLMPredictor,
    ServiceContext,
    GPTSQLStructStoreIndex,
    SQLDatabase
)

ImportError: cannot import name 'LLMPredictor' from 'llama_index' (unknown location)